In [106]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

# 무신사뷰티

In [107]:
# 데이터 불러오기
musinsa = pd.read_csv('통합_무신사_데이터.csv')

# 데이터셋 복제
musinsa_df = musinsa.copy(deep=True)

In [108]:
# 데이터셋 기본 정보 조회
print(musinsa_df.info())
print(musinsa_df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   순위      400 non-null    int64 
 1   카테고리    400 non-null    object
 2   중분류     400 non-null    object
 3   소분류     400 non-null    object
 4   브랜드     400 non-null    object
 5   제품명     400 non-null    object
 6   평점      400 non-null    object
 7   후기 수    400 non-null    object
 8   가격      400 non-null    object
 9   URL     400 non-null    object
dtypes: int64(1), object(9)
memory usage: 31.4+ KB
None
   순위 카테고리     중분류   소분류    브랜드  \
0   1  클렌징  페이셜클렌징  클렌징폼  이니스프리   
1   2  클렌징  페이셜클렌징  클렌징폼   비플레인   
2   3  클렌징  페이셜클렌징  클렌징폼  에스네이처   

                                                 제품명   평점   후기 수      가격  \
0                           화산송이 모공 바하 클렌징 폼 150g 2개  4.9  7,915  18,200   
1              [대용량]녹두 약산성 클렌징 폼 160ml 2개 (+40ml 증정)  4.9  2,870  31,200   
2  아쿠아 라이스 약산성 클렌징폼 160ml 2개입 [증정: 클렌징

## 기본 전처리

In [109]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 1) 후기 수 & 가격 컬럼의 쉼표 없애기
# + 평점 컬럼의 '별점 없음' 을 0으로 변경

for i in range(len(musinsa_df)):
  score = musinsa_df['평점'][i]
  review = musinsa_df['후기 수'][i]
  price = musinsa_df['가격'][i]
  
  score_result = score.replace('별점 없음', '0')
  review_result = review.replace(',', '')
  price_result = price.replace(',', '')
  
  musinsa_df.loc[i, '평점'] = score_result
  musinsa_df.loc[i, '후기 수'] = review_result
  musinsa_df.loc[i, '가격'] = price_result

print(musinsa_df.loc[:10, ['평점', '후기 수', '가격']])

     평점  후기 수     가격
0   4.9  7915  18200
1   4.9  2870  31200
2   4.9   630  31900
3   4.9  3707  23000
4   4.9  1215  23000
5   4.9   990  23000
6   4.9  1004  12900
7   4.8   217  29000
8   4.9  5188  19500
9   4.9  1376  17400
10  4.8  1930  21000


In [110]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 2) 컬럼 타입 변경

musinsa_df[['평점', '후기 수', '가격']] = musinsa_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)

print(musinsa_df[['평점', '후기 수', '가격']].dtypes)
print(musinsa_df.loc[:5, ['평점', '후기 수', '가격']])

평점      float64
후기 수      int64
가격        int64
dtype: object
    평점  후기 수     가격
0  4.9  7915  18200
1  4.9  2870  31200
2  4.9   630  31900
3  4.9  3707  23000
4  4.9  1215  23000
5  4.9   990  23000


In [111]:
# 2. 컬럼 이름 변경
musinsa_df.rename(columns = {'순위':'중분류_순위', '카테고리':'대분류'}, inplace=True)
musinsa_df.columns

Index(['중분류_순위', '대분류', '중분류', '소분류', '브랜드', '제품명', '평점', '후기 수', '가격', 'URL'], dtype='object')

In [112]:
# 3. 플랫폼 컬럼 추가 및 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류', '소분류', '평점', '후기 수', '정가', '할인가'
musinsa_df['플랫폼'] = '무신사뷰티'

before_a = ['플랫폼']
a = ['브랜드', '제품명']
after_a = musinsa_df.drop(columns = {'플랫폼', '브랜드', '제품명'}).columns[:].to_list()

reorder = before_a + a + after_a
musinsa_df = musinsa_df[reorder]

musinsa_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류', '소분류', '평점', '후기 수', '가격',
       'URL'],
      dtype='object')

## 대분류 및 중분류 통합

In [113]:
# 1. 중분류1 & 중분류2 컬럼 생성
musinsa_df['중분류1'] = ''
print(musinsa_df['중분류1'].head())

musinsa_df['중분류2'] = ''
print(musinsa_df['중분류2'].head())

0    
1    
2    
3    
4    
Name: 중분류1, dtype: object
0    
1    
2    
3    
4    
Name: 중분류2, dtype: object


In [114]:
# 기존 분류별 unique 값 확인
print(musinsa_df['대분류'].unique())
print(musinsa_df['중분류'].unique())
print(musinsa_df['소분류'].unique())

['클렌징' '스킨케어' '썬케어' '메이크업']
['페이셜클렌징' '메이크업클렌징' '필링/스크럽' '립&아이 리무버' '에센스/세럼/앰플' '기획세트' '마스크팩' '크림'
 '패드' '로션' '미스트' '스킨/토너' '올인원' '스팟케어' '선크림' '선세럼/젤' '선쿠션/파우더' '선스틱/선스프레이'
 '태닝/애프터선' '선케어세트' '립메이크업' '베이스메이크업' '아이메이크업']
['클렌징폼' '클렌징로션/크림' '클렌징오일' '필링' '없음' '클렌징패드' '페이스 스크럽' '클렌징워터' '바디 스크럽'
 '클렌징젤' '카테고리 없음' '페이셜팩' '코팩/패치' '선세럼' '선스틱' '애프터선' '태닝' '선젤' '립틴트' '쿠션'
 '파운데이션' '하이라이터' '립밤' '립라이너' '셰이딩/컨투어링' '파우더/콤팩트' '아이라이너' '블러셔' '컨실러'
 '메이크업 베이스' '립글로스' '아이섀도' '립스틱' '마스카라' 'BB크림/CC크림' '아이브로' '메이크업 픽서']


In [115]:
# 기타 컬럼 소속 예정 값의 대분류 값 확인
cond1 = musinsa_df['중분류'] == '기획세트'
print(musinsa_df.loc[cond1, '대분류'].unique()) # 스킨케어

cond2 = musinsa_df['중분류'] == '립&아이 리무버'
print(musinsa_df.loc[cond2, '대분류'].unique()) # 클렌징 -> 기타에서 제외

cond3 = musinsa_df['중분류'] == '미스트'
print(musinsa_df.loc[cond3, '대분류'].unique()) # 스킨케어

cond4 = musinsa_df['중분류'] == '올인원'
print(musinsa_df.loc[cond4, '대분류'].unique()) # 스킨케어 

cond5 = musinsa_df['중분류'] == '스팟케어'
print(musinsa_df.loc[cond5, '대분류'].unique()) # 스킨케어 

['스킨케어']
['클렌징']
['스킨케어']
['스킨케어']
['스킨케어']


In [116]:
# 중분류2
# 스킨케어 - 마스크팩, 클렌징, 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림, 기타
# 베이스메이크업 - 선케어, 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 색조메이크업 - 립메이크업, 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제

skin_list = ['패드', '스킨/토너']
skin = '|'.join(skin_list)
cond_skin = musinsa_df['중분류'].str.contains(skin)

found_list = ['파운데이션', '쿠션', 'BB', 'CC']
found  = '|'.join(found_list)
cond_found = musinsa_df['소분류'].str.contains(found)

fixer_list = ['파우더', '팩트', '픽서']
fixer = '|'.join(fixer_list)
cond_fixer = musinsa_df['소분류'].str.contains(fixer)

shading_list = ['블러셔', '쉐이딩', '셰이딩', '하이라이터']
shading = '|'.join(shading_list)
cond_shading = musinsa_df['소분류'].str.contains(shading)

primer_list = ['프라이머', '베이스']
primer = '|'.join(primer_list)
cond_primer = musinsa_df['소분류'].str.contains(primer)

for i in range(len(musinsa_df)):
  if cond_skin[i] == True:
    musinsa_df.loc[i, '중분류2'] = '토너/토너패드/스킨'
  
  elif cond_found[i] == True:
    musinsa_df.loc[i, '중분류2'] = '파운데이션/쿠션/BB/CC'

  elif cond_fixer[i] == True:
    musinsa_df.loc[i, '중분류2'] = '파우더/팩트/픽서'
  
  elif cond_shading[i] == True:
    musinsa_df.loc[i, '중분류2'] = '블러셔/쉐이딩/하이라이터'

  elif cond_primer[i] == True:
    musinsa_df.loc[i, '중분류2'] = '프라이머/베이스'
      
  else:
    if musinsa_df['대분류'][i] == '클렌징':
      musinsa_df.loc[i, '중분류2'] = '클렌징'

    elif musinsa_df['중분류'][i] == '에센스/세럼/앰플':
      musinsa_df.loc[i, '중분류2'] = '에센스/세럼/앰플'

    elif musinsa_df['중분류'][i] == '마스크팩':
      musinsa_df.loc[i, '중분류2'] = '마스크팩'
      
    elif musinsa_df['중분류'][i] == '크림':
      musinsa_df.loc[i, '중분류2'] = '크림'

    elif musinsa_df['중분류'][i] == '로션':
      musinsa_df.loc[i, '중분류2'] = '로션'

    elif musinsa_df['대분류'][i] == '썬케어':
      musinsa_df.loc[i, '중분류2'] = '선케어'

    elif musinsa_df['소분류'][i] == '컨실러':
      musinsa_df.loc[i, '중분류2'] = '컨실러'

    elif musinsa_df['중분류'][i] == '립메이크업':
      musinsa_df.loc[i, '중분류2'] = '립메이크업'
      
    elif musinsa_df['소분류'][i] == '아이라이너':
      musinsa_df.loc[i, '중분류2'] = '아이라이너'

    elif musinsa_df['소분류'][i] == '아이섀도':
      musinsa_df.loc[i, '중분류2'] = '아이섀도우'
      
    elif musinsa_df['소분류'][i] == '아이브로':
      musinsa_df.loc[i, '중분류2'] = '아이브로우'

    elif musinsa_df['소분류'][i] == '마스카라':
      musinsa_df.loc[i, '중분류2'] = '마스카라'
      
    else: # 기획세트
      musinsa_df.loc[i, '중분류2'] = '기타'

In [117]:
print(musinsa_df['중분류2'].unique())
print(musinsa_df['중분류2'].nunique())
musinsa_df.loc[:, ['대분류', '중분류', '소분류', '중분류2']]

['클렌징' '에센스/세럼/앰플' '기타' '마스크팩' '크림' '토너/토너패드/스킨' '로션' '선케어' '립메이크업'
 '파운데이션/쿠션/BB/CC' '블러셔/쉐이딩/하이라이터' '파우더/팩트/픽서' '아이라이너' '컨실러' '프라이머/베이스'
 '아이섀도우' '마스카라' '아이브로우']
18


,대분류,중분류,소분류,중분류2
0,클렌징,페이셜클렌징,클렌징폼,클렌징
1,클렌징,페이셜클렌징,클렌징폼,클렌징
2,클렌징,페이셜클렌징,클렌징폼,클렌징
3,클렌징,페이셜클렌징,클렌징폼,클렌징
4,클렌징,메이크업클렌징,클렌징로션/크림,클렌징
...,...,...,...,...
395,메이크업,베이스메이크업,BB크림/CC크림,파운데이션/쿠션/BB/CC
396,메이크업,립메이크업,립틴트,립메이크업
397,메이크업,베이스메이크업,블러셔,블러셔/쉐이딩/하이라이터
398,메이크업,립메이크업,립틴트,립메이크업


In [118]:
# 중분류1 - 중분류2
# 스킨케어 - 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림
# 베이스메이크업 - 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 아이메이크업 - 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
# 클렌징 / # 마스크팩 / # 기타 / # 선케어 / # 립메이크업

skincare_list = ['토너/토너패드/스킨', '에센스/세럼/앰플', '로션', '크림']
skincare = '|'.join(skincare_list)
cond_skincare = musinsa_df['중분류2'].str.contains(skincare)

base_list = ['파운데이션/쿠션/BB/CC', '파우더/팩트/픽서', '블러셔/쉐이딩/하이라이터', '프라이머/베이스', '컨실러']
base = '|'.join(base_list)
cond_base = musinsa_df['중분류2'].str.contains(base)

eye_list = ['아이섀도우', '아이라이너', '아이브로우', '마스카라', '속눈썹 영양제']
eye = '|'.join(eye_list)
cond_eye = musinsa_df['중분류2'].str.contains(eye)

for i in range(len(musinsa_df)):
  if cond_skincare[i] == True:
    musinsa_df.loc[i, '중분류1'] = '스킨케어'
  
  elif cond_base[i] == True:
    musinsa_df.loc[i, '중분류1'] = '베이스메이크업'

  elif cond_eye[i] == True:
    musinsa_df.loc[i, '중분류1'] = '아이메이크업'
      
  else:
    if musinsa_df['중분류2'][i] == '클렌징':
      musinsa_df.loc[i, '중분류1'] = '클렌징'

    elif musinsa_df['중분류2'][i] == '마스크팩':
      musinsa_df.loc[i, '중분류1'] = '마스크팩'
      
    elif musinsa_df['중분류2'][i] == '기타':
      musinsa_df.loc[i, '중분류1'] = '기타'
      
    elif musinsa_df['중분류2'][i] == '선케어':
      musinsa_df.loc[i, '중분류1'] = '선케어'
    
    elif musinsa_df['중분류2'][i] == '립메이크업':
      musinsa_df.loc[i, '중분류1'] = '립메이크업'

In [119]:
print(musinsa_df['중분류1'].unique())
print(musinsa_df['중분류1'].nunique())
musinsa_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['클렌징' '스킨케어' '기타' '마스크팩' '선케어' '립메이크업' '베이스메이크업' '아이메이크업']
8


,대분류,중분류,소분류,중분류1,중분류2
0,클렌징,페이셜클렌징,클렌징폼,클렌징,클렌징
1,클렌징,페이셜클렌징,클렌징폼,클렌징,클렌징
2,클렌징,페이셜클렌징,클렌징폼,클렌징,클렌징
3,클렌징,페이셜클렌징,클렌징폼,클렌징,클렌징
4,클렌징,메이크업클렌징,클렌징로션/크림,클렌징,클렌징
...,...,...,...,...,...
395,메이크업,베이스메이크업,BB크림/CC크림,베이스메이크업,파운데이션/쿠션/BB/CC
396,메이크업,립메이크업,립틴트,립메이크업,립메이크업
397,메이크업,베이스메이크업,블러셔,베이스메이크업,블러셔/쉐이딩/하이라이터
398,메이크업,립메이크업,립틴트,립메이크업,립메이크업


In [120]:
# 대분류 - 중분류1
# 스킨케어 - 마스크팩, 클렌징, 기타, 스킨케어
# 베이스메이크업 - 선케어, 베이스메이크업
# 색조메이크업 - 립메이크업, 아이메이크업

skinclass_list = ['마스크팩', '클렌징', '기타', '스킨케어']
skinclass = '|'.join(skinclass_list)
cond_skinclass = musinsa_df['중분류1'].str.contains(skinclass)

baseclass_list = ['선케어', '베이스메이크업']
baseclass = '|'.join(baseclass_list)
cond_baseclass = musinsa_df['중분류1'].str.contains(baseclass)

colorclass_list = ['아이메이크업', '립메이크업']
colorclass = '|'.join(colorclass_list)
cond_colorclass = musinsa_df['중분류1'].str.contains(colorclass)

for i in range(len(musinsa_df)):
  if cond_skinclass[i] == True:
    musinsa_df.loc[i, '대분류'] = '스킨케어'
  
  elif cond_baseclass[i] == True:
    musinsa_df.loc[i, '대분류'] = '베이스메이크업'

  elif cond_colorclass[i] == True:
    musinsa_df.loc[i, '대분류'] = '색조메이크업'

In [121]:
# 대분류 고유값 확인 
print(musinsa_df['대분류'].unique())
print(musinsa_df['대분류'].nunique())
musinsa_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '베이스메이크업' '색조메이크업']
3


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,페이셜클렌징,클렌징폼,클렌징,클렌징
1,스킨케어,페이셜클렌징,클렌징폼,클렌징,클렌징
2,스킨케어,페이셜클렌징,클렌징폼,클렌징,클렌징
3,스킨케어,페이셜클렌징,클렌징폼,클렌징,클렌징
4,스킨케어,메이크업클렌징,클렌징로션/크림,클렌징,클렌징
...,...,...,...,...,...
395,베이스메이크업,베이스메이크업,BB크림/CC크림,베이스메이크업,파운데이션/쿠션/BB/CC
396,색조메이크업,립메이크업,립틴트,립메이크업,립메이크업
397,베이스메이크업,베이스메이크업,블러셔,베이스메이크업,블러셔/쉐이딩/하이라이터
398,색조메이크업,립메이크업,립틴트,립메이크업,립메이크업


## 립메이크업 중분류 세분화

In [122]:
# musinsa_df
musinsa_df.head()

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류,소분류,평점,후기 수,가격,URL,중분류1,중분류2
0,무신사뷰티,이니스프리,화산송이 모공 바하 클렌징 폼 150g 2개,1,스킨케어,페이셜클렌징,클렌징폼,4.9,7915,18200,https://www.musinsa.com/products/3670796,클렌징,클렌징
1,무신사뷰티,비플레인,[대용량]녹두 약산성 클렌징 폼 160ml 2개 (+40ml 증정),2,스킨케어,페이셜클렌징,클렌징폼,4.9,2870,31200,https://www.musinsa.com/products/3138771,클렌징,클렌징
2,무신사뷰티,에스네이처,아쿠아 라이스 약산성 클렌징폼 160ml 2개입 [증정: 클렌징 오일 13ml + ...,3,스킨케어,페이셜클렌징,클렌징폼,4.9,630,31900,https://www.musinsa.com/products/4566395,클렌징,클렌징
3,무신사뷰티,토리든,[2개묶음] 다이브인 저분자 히알루론산 클렌징 폼 150ml x 2개 [총 300ml],4,스킨케어,페이셜클렌징,클렌징폼,4.9,3707,23000,https://www.musinsa.com/products/3312180,클렌징,클렌징
4,무신사뷰티,바닐라코,클린잇제로 오리지널 클렌징밤 대용량 180ml,5,스킨케어,메이크업클렌징,클렌징로션/크림,4.9,1215,23000,https://www.musinsa.com/products/3994035,클렌징,클렌징


In [123]:
# 중분류2 복제
musinsa_df['중분류3'] = musinsa_df['중분류2']
musinsa_df['중분류3'].head()

0    클렌징
1    클렌징
2    클렌징
3    클렌징
4    클렌징
Name: 중분류3, dtype: object

In [124]:
# '중분류2 = 립메이크업 제품' 소분류 카테고리 고유값 확인
cond = musinsa_df['중분류2'] == '립메이크업'
musinsa_df.loc[cond, '소분류'].unique()

array(['립틴트', '립밤', '립라이너', '립글로스', '립스틱'], dtype=object)

In [125]:
# 립메이크업- 립글로스/립밤/립스틱/립라이너/립틴트 

for i in range(len(musinsa_df)):
  if musinsa_df['소분류'][i] == '립글로스':
    musinsa_df.loc[i, '중분류3'] = '립글로스'

  elif musinsa_df['소분류'][i] == '립밤':
    musinsa_df.loc[i, '중분류3'] = '립밤'
    
  elif musinsa_df['소분류'][i] == '립스틱':
    musinsa_df.loc[i, '중분류3'] = '립스틱'
    
  elif musinsa_df['소분류'][i] == '립라이너':
    musinsa_df.loc[i, '중분류3'] = '립라이너'

  elif musinsa_df['소분류'][i] == '립틴트':
    musinsa_df.loc[i, '중분류3'] = '립틴트'

In [126]:
# 중분류3 입력값 확인
cond = musinsa_df['중분류2'] == '립메이크업'
print(musinsa_df.loc[cond, '중분류3'].unique())
print(musinsa_df.loc[cond, '중분류3'].nunique())
musinsa_df.loc[cond, ['소분류', '중분류3']].head()

['립틴트' '립밤' '립라이너' '립글로스' '립스틱']
5


,소분류,중분류3
300,립틴트,립틴트
302,립틴트,립틴트
308,립밤,립밤
310,립라이너,립라이너
312,립밤,립밤


## 데이터셋 정리

In [127]:
# 기존 카테고리 지우기
musinsa_df.drop(columns={'중분류', '소분류'}, axis=1, inplace=True)
musinsa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     400 non-null    object 
 1   브랜드     400 non-null    object 
 2   제품명     400 non-null    object 
 3   중분류_순위  400 non-null    int64  
 4   대분류     400 non-null    object 
 5   평점      400 non-null    float64
 6   후기 수    400 non-null    int64  
 7   가격      400 non-null    int64  
 8   URL     400 non-null    object 
 9   중분류1    400 non-null    object 
 10  중분류2    400 non-null    object 
 11  중분류3    400 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 37.6+ KB


In [128]:
# 컬럼 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수', '정가', '할인가'
a = ['중분류1', '중분류2', '중분류3']

before_a = musinsa_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[:5].to_list()
after_a = musinsa_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[5:].to_list()

reorder = before_a + a + after_a
musinsa_df = musinsa_df[reorder]

musinsa_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점',
       '후기 수', '가격', 'URL'],
      dtype='object')

In [129]:
# 최종 데이터셋 확인
print(musinsa_df.info())
musinsa_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     400 non-null    object 
 1   브랜드     400 non-null    object 
 2   제품명     400 non-null    object 
 3   중분류_순위  400 non-null    int64  
 4   대분류     400 non-null    object 
 5   중분류1    400 non-null    object 
 6   중분류2    400 non-null    object 
 7   중분류3    400 non-null    object 
 8   평점      400 non-null    float64
 9   후기 수    400 non-null    int64  
 10  가격      400 non-null    int64  
 11  URL     400 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 37.6+ KB
None


,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격,URL
0,무신사뷰티,이니스프리,화산송이 모공 바하 클렌징 폼 150g 2개,1,스킨케어,클렌징,클렌징,클렌징,4.9,7915,18200,https://www.musinsa.com/products/3670796
1,무신사뷰티,비플레인,[대용량]녹두 약산성 클렌징 폼 160ml 2개 (+40ml 증정),2,스킨케어,클렌징,클렌징,클렌징,4.9,2870,31200,https://www.musinsa.com/products/3138771
2,무신사뷰티,에스네이처,아쿠아 라이스 약산성 클렌징폼 160ml 2개입 [증정: 클렌징 오일 13ml + ...,3,스킨케어,클렌징,클렌징,클렌징,4.9,630,31900,https://www.musinsa.com/products/4566395
3,무신사뷰티,토리든,[2개묶음] 다이브인 저분자 히알루론산 클렌징 폼 150ml x 2개 [총 300ml],4,스킨케어,클렌징,클렌징,클렌징,4.9,3707,23000,https://www.musinsa.com/products/3312180
4,무신사뷰티,바닐라코,클린잇제로 오리지널 클렌징밤 대용량 180ml,5,스킨케어,클렌징,클렌징,클렌징,4.9,1215,23000,https://www.musinsa.com/products/3994035


In [130]:
# 최종 데이터셋 저장
# musinsa_df.to_csv('musinsa_preprocessing.csv', index=False, encoding="utf-8-sig")